In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

In [2]:
mytable = np.array([[5, 15],[3,30]])
mytable

array([[ 5, 15],
       [ 3, 30]])

#### The methods below refer to the statsmodel package. 

##### Learn to read the docs! https://www.statsmodels.org/stable/generated/statsmodels.stats.contingency_tables.Table2x2.html

In [3]:
result = sm.stats.Table2x2(mytable)
result.oddsratio

3.3333333333333335

In [4]:
result.oddsratio_confint()

(0.7006053806126253, 15.859300283128432)

In [5]:
result.oddsratio_pvalue()

0.13031366333302763

## Logistic Regression 

### Read the data

In [6]:
raw_data = pd.read_csv(os.path.expanduser("~/gitRepos/cse5243/course_materials/lecture_examples/LogisticRegressionData.csv"))
raw_data

,Sick,Age,Sex,Smoker
0,0,24,0,0
1,1,40,1,1
2,0,23,0,0
3,1,39,0,1
4,0,26,0,0
5,1,45,1,1
6,1,42,1,0
7,1,39,1,1
8,0,23,1,0
9,0,38,0,0


In [7]:
raw_data.shape

(20, 4)

### Logistic Reression of One Variable

In [8]:
my_model = sm.Logit(raw_data['Sick'], sm.add_constant(raw_data['Smoker'])).fit() 
print(my_model.summary())

#be sure to check out the formulaic version as well if you're familiar with R: https://www.statsmodels.org/dev/example_formulas.html

Optimization terminated successfully.
         Current function value: 0.479248
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   Sick   No. Observations:                   20
Model:                          Logit   Df Residuals:                       18
Method:                           MLE   Df Model:                            1
Date:                Thu, 26 Jan 2023   Pseudo R-squ.:                  0.3036
Time:                        16:23:16   Log-Likelihood:                -9.5850
converged:                       True   LL-Null:                       -13.763
Covariance Type:            nonrobust   LLR p-value:                  0.003845
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9808      0.677     -1.449      0.147      -2.308       0.346
Smoker         3.0603      1.

In [9]:
predicted_probs = my_model.predict(sm.add_constant(raw_data['Smoker']))
predicted_probs

0     0.272727
1     0.888889
2     0.272727
3     0.888889
4     0.272727
5     0.888889
6     0.272727
7     0.888889
8     0.272727
9     0.272727
10    0.272727
11    0.272727
12    0.888889
13    0.272727
14    0.272727
15    0.272727
16    0.888889
17    0.888889
18    0.888889
19    0.888889
dtype: float64

In [10]:
def table(predicted_probs, labels, cutoff):
    """ Replacement for R's table funcion. """
    predicted_outcome = (predicted_probs > cutoff).astype(int)
    df = pd.DataFrame({"predicted_outcome": predicted_outcome, "actual_outcome": labels})
    return pd.crosstab(index=df["actual_outcome"], columns=df["predicted_outcome"], margins=False)

table(predicted_probs, raw_data["Sick"], cutoff=0.5)

predicted_outcome,0,1
actual_outcome,,
0,8,1
1,3,8


### Logistic Regression of Multiple Variables

In [11]:
my_model_full = sm.Logit(raw_data['Sick'], sm.add_constant(raw_data[['Smoker', 'Age', 'Sex']])).fit() 
print(my_model_full.summary())

Optimization terminated successfully.
         Current function value: 0.285272
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                   Sick   No. Observations:                   20
Model:                          Logit   Df Residuals:                       16
Method:                           MLE   Df Model:                            3
Date:                Thu, 26 Jan 2023   Pseudo R-squ.:                  0.5854
Time:                        16:23:21   Log-Likelihood:                -5.7054
converged:                       True   LL-Null:                       -13.763
Covariance Type:            nonrobust   LLR p-value:                  0.001074
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -13.1850      8.527     -1.546      0.122     -29.897       3.527
Smoker         1.1918      1.

In [12]:
np.exp(5*-0.9784 )

0.007506394662864678

### What about all you sklearn folks?
#### Or why it's important to actually know what you're doing AND read the docs!

In [15]:
from sklearn.linear_model import LogisticRegression
sk_model = LogisticRegression(fit_intercept = True, penalty = 'none').fit(raw_data[['Smoker', 'Age', 'Sex']], raw_data['Sick'])

###Note the difference. You must specificy, the penalty (such as L1, L2, elasticnet, none)
### Other parameters possible: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
### Also Syntax is different.
### Moral of the story: Understand the THEORY.. Then figure out the code. 


In [16]:
sk_model.coef_

array([[ 1.19174798,  0.36722857, -0.9783657 ]])

In [17]:
sk_model.intercept_

array([-13.1849339])

#### But do they give the same results?

In [18]:
predicted_probs_stats_models = my_model_full.predict( sm.add_constant(raw_data[['Smoker', 'Age', 'Sex']]))
predicted_probs_sklearn = sk_model.predict_proba(raw_data[['Smoker', 'Age', 'Sex']])

In [19]:
predicted_probs_stats_models ## This shows the probability of a 1

0     0.012475
1     0.847819
2     0.008674
3     0.911229
4     0.025655
5     0.972179
6     0.779078
7     0.794189
8     0.003279
9     0.683467
10    0.014089
11    0.757125
12    0.920712
13    0.012475
14    0.683467
15    0.020215
16    0.936788
17    0.920712
18    0.727730
19    0.968642
dtype: float64

In [20]:
predicted_probs_sklearn ## This shows the probabiltiy of [0,1]

array([[0.98752467, 0.01247533],
       [0.15218168, 0.84781832],
       [0.99132568, 0.00867432],
       [0.08877147, 0.91122853],
       [0.9743441 , 0.0256559 ],
       [0.0278214 , 0.9721786 ],
       [0.22092218, 0.77907782],
       [0.20581108, 0.79418892],
       [0.99672136, 0.00327864],
       [0.31653279, 0.68346721],
       [0.98591039, 0.01408961],
       [0.24287519, 0.75712481],
       [0.07928888, 0.92071112],
       [0.98752467, 0.01247533],
       [0.31653279, 0.68346721],
       [0.97978483, 0.02021517],
       [0.06321235, 0.93678765],
       [0.07928888, 0.92071112],
       [0.27227034, 0.72772966],
       [0.03135835, 0.96864165]])

In [21]:
predicted_probs_stats_models - predicted_probs_sklearn[:, 1]

0    -2.888780e-07
1     4.769032e-07
2    -2.168378e-07
3     9.227004e-07
4    -4.977668e-07
5     3.390727e-07
6    -1.898509e-07
7     3.151823e-07
8    -1.133782e-07
9     2.819764e-07
10   -3.837108e-07
11    5.647905e-07
12    5.279722e-07
13   -2.888780e-07
14    2.819764e-07
15   -5.120945e-07
16    7.801594e-07
17    5.279722e-07
18    3.175547e-08
19    5.075860e-07
dtype: float64